In [1]:
import sys
sys.path.insert(0, '..')  # Add the parent directory (project root directory) to the Python path

In [2]:
from utils.processing import filtered_df

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('../data/bond_etf_yield.csv', index_col=0)
df

,Name,Yield to Maturity,Expected Annualized Std Dev,P/E Ratio,Yield to Volatility,Yield ZScore,Date
Ticker,,,,,,,
AGG,iShares Core U.S. Aggregate Bond ETF,0.0511,0.0681,19.57,0.75,0.34,08-18-2023
TLT,iShares 20+ Year Treasury Bond ETF,0.0450,0.1646,22.22,0.27,0.71,08-18-2023
LQD,iShares iBoxx $ Investment Grade Corporate Bon...,0.0575,0.0909,17.39,0.63,0.05,08-18-2023
MUB,iShares National Muni Bond ETF,0.0362,0.0400,27.62,0.90,1.24,08-18-2023
IEF,iShares 7-10 Year Treasury Bond ETF,0.0428,0.0858,23.36,0.50,0.84,08-18-2023
...,...,...,...,...,...,...,...
MBSD,FLEXSHARES DISCIPLINED DURATION MBS INDEX FUND,0.0567,0.0571,17.64,0.99,0.00,08-21-2023
RAVI,FLEXSHARES ULTRA-SHORT INCOME FUND,0.0591,0.0148,16.92,3.99,0.15,08-21-2023
SKOR,FLEXSHARES CREDIT-SCORED US CORPORATE BOND IND...,0.0592,0.0489,16.89,1.21,0.15,08-21-2023


In [5]:
df.sort_values(by='Yield to Maturity', ascending=False).dropna()

,Name,Yield to Maturity,Expected Annualized Std Dev,P/E Ratio,Yield to Volatility,Yield ZScore,Date
Ticker,,,,,,,
PGHY,Invesco Global Short Term High Yield Bond ETF,0.1021,0.0716,9.79,1.43,2.76,08-18-2023
JBBB,B-BBB CLO ETF,0.0998,0.0426,10.02,2.34,2.62,08-21-2023
HYGV,FLEXSHARES HIGH YIELD VALUE-SCORED BOND INDEX ...,0.0998,0.0665,10.02,1.50,2.62,08-21-2023
EMHY,iShares J.P. Morgan EM High Yield Bond ETF,0.0984,0.0782,10.16,1.26,2.53,08-18-2023
BKLN,Invesco Senior Loan ETF,0.0963,0.0402,10.38,2.40,2.40,08-18-2023
...,...,...,...,...,...,...,...
BBIP,JPMorgan BetaBuilders U.S. TIPS 0-5 Year ETF,0.0254,0.0242,39.37,1.05,1.90,07-31-2023
TIP,iShares TIPS Bond ETF,0.0243,0.0617,41.15,0.39,1.97,08-18-2023
PBTP,Invesco PureBeta℠ 0-5 Yr US TIPS ETF,0.0227,0.0370,44.05,0.61,2.06,08-18-2023


In [6]:
df.loc['HYGV']

Name                           FLEXSHARES HIGH YIELD VALUE-SCORED BOND INDEX ...
Yield to Maturity                                                         0.0998
Expected Annualized Std Dev                                               0.0665
P/E Ratio                                                                  10.02
Yield to Volatility                                                          1.5
Yield ZScore                                                                2.62
Date                                                                  08-21-2023
Name: HYGV, dtype: object

In [7]:
filtered_df(df)

,Name,Yield to Maturity,Expected Annualized Std Dev,P/E Ratio,Yield to Volatility,Yield ZScore,Date
Ticker,,,,,,,
JAAA,AAA CLO ETF,7.39%,1.67%,13.53,4.43,1.04,08-21-2023
STOT,SPDR® DoubleLine® Short Duration Total Return ...,6.40%,1.87%,15.62,3.42,0.44,08-21-2023
NEAR,BlackRock Short Maturity Bond ETF,6.29%,1.13%,15.90,5.57,0.38,08-18-2023
FLOT,iShares Floating Rate Bond ETF,6.24%,1.22%,16.03,5.11,0.35,08-18-2023
GSY,Invesco Ultra Short Duration ETF,6.19%,1.42%,16.16,4.36,0.32,08-18-2023
FLRN,SPDR® Bloomberg Investment Grade Floating Rate...,6.13%,1.62%,16.31,3.78,0.28,08-21-2023
JPST,JPMorgan Ultra-Short Income ETF,6.08%,1.19%,16.45,5.11,0.25,08-21-2023
IBDP,iShares® iBonds® Dec 2024 Term Corporate ETF,5.92%,1.43%,16.89,4.14,0.15,08-18-2023
SPSB,SPDR® Portfolio Short Term Corporate Bond ETF,5.91%,1.68%,16.92,3.52,0.15,08-21-2023
